In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.layers as L
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image


In [ ]:
# Set seeds for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)


In [ ]:
#Define image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 16

In [ ]:
# Load the inbuilt Stanford Dogs dataset from TensorFlow Datasets
(raw_train, raw_validation), metadata = tfds.load(
    'stanford_dogs',
    split=['train[:80%]', 'train[20%:]'],  # 80% for training, 20% for validation
    with_info=True,
    as_supervised=True,  # Returns (image, label) pairs
 )

In [ ]:
# Load dataset info
dataset_info = tfds.builder('stanford_dogs').info

# Extract class names
class_names = dataset_info.features['label'].names
print(class_names)

['n02085620-chihuahua', 'n02085782-japanese_spaniel', 'n02085936-maltese_dog', 'n02086079-pekinese', 'n02086240-shih-tzu', 'n02086646-blenheim_spaniel', 'n02086910-papillon', 'n02087046-toy_terrier', 'n02087394-rhodesian_ridgeback', 'n02088094-afghan_hound', 'n02088238-basset', 'n02088364-beagle', 'n02088466-bloodhound', 'n02088632-bluetick', 'n02089078-black-and-tan_coonhound', 'n02089867-walker_hound', 'n02089973-english_foxhound', 'n02090379-redbone', 'n02090622-borzoi', 'n02090721-irish_wolfhound', 'n02091032-italian_greyhound', 'n02091134-whippet', 'n02091244-ibizan_hound', 'n02091467-norwegian_elkhound', 'n02091635-otterhound', 'n02091831-saluki', 'n02092002-scottish_deerhound', 'n02092339-weimaraner', 'n02093256-staffordshire_bullterrier', 'n02093428-american_staffordshire_terrier', 'n02093647-bedlington_terrier', 'n02093754-border_terrier', 'n02093859-kerry_blue_terrier', 'n02093991-irish_terrier', 'n02094114-norfolk_terrier', 'n02094258-norwich_terrier', 'n02094433-yorkshire_t

In [ ]:
# # Define the Learning Rate Scheduler (Exponential Decay)
# initial_learning_rate = 0.01
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=initial_learning_rate,
#     decay_steps=100000,  # Number of steps after which the learning rate will decay
#     decay_rate=0.96,     # The rate at which the learning rate decays
#     staircase=True)

# Monitor validation loss and reduce the learning rate when it stops improving
lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,  # Reduce learning rate by a factor of 0.5
    patience=3,  # Number of epochs with no improvement before reducing LR
    verbose=1,  # Print a message when learning rate is reduced
    min_lr=1e-6  # Minimum learning rate
)


## Data Preprocessing

In [ ]:
# Convert TFDS dataset to NumPy arrays
def tfds_to_numpy(dataset):
    images = []
    labels = []
    for img, label in tfds.as_numpy(dataset):
        images.append(tf.image.resize(img, (img_height, img_width)).numpy())
        labels.append(label)
    return np.array(images), np.array(labels)


train_images, train_labels = tfds_to_numpy(raw_train)
val_images, val_labels = tfds_to_numpy(raw_validation)

In [ ]:
# Define custom augmentation function
def random_crop(img):
    img = tf.image.random_crop(img, size=[img_height, img_width, 3])  # Random crop to required size
    return img

# Data Augmentation: Apply random transformations to the image for better generalization
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=45,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Randomly shear images
    zoom_range=0.2,  # Randomly zoom images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest',  # Fill missing pixels after transformations
    brightness_range=[0.8, 1.2],
    channel_shift_range=50.0, # change range of colors
    preprocessing_function=random_crop)


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for validation

In [ ]:
# Create data generators
train_generator = train_datagen.flow(
    train_images, train_labels, batch_size=batch_size
)

validation_generator = validation_datagen.flow(
    val_images, val_labels, batch_size=batch_size
)

In [ ]:
print(len(train_generator))
print(len(validation_generator))


## Photo Preview

In [ ]:
# Visualize a batch of images after augmentation
batch = next(train_generator)  # Get one batch of data
images, labels = batch  # Unpack the images and labels from the batch

plt.figure(figsize=(10, 10))

# Plot 16 images in a 4x4 grid
for i in range(16):  # Loop through the batch of 16 images
    plt.subplot(4, 4, i + 1)
    plt.imshow(images[i])  # Use the augmented image
    breed_name = label_map[labels[i]]  # Map the label to the breed name
    breed_name = breed_name.split('-')[-1]  # Only use the breed name (ignore the label number)
    plt.title(f"{breed_name}")
    plt.axis('off')

plt.tight_layout()
plt.show()


## Import ResNet50

In [ ]:
# Load the EfficientNetB0 architecture from Keras, without pre-trained weights
base_model = tf.keras.applications.ResNet50(
    include_top=False,  # Exclude the final fully connected layer
    weights='imagenet',  # Load pretrained weights from ImageNet
    input_shape=(img_height, img_width, 3)
)

In [ ]:
# Freeze the base_model layers (don't train them)
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom layers on top
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)  # Apply global average pooling to the output
x = layers.Dense(256, activation='relu')(x)  # Add a fully connected layer
x = layers.Dense(120, activation='softmax')(x)  # Output layer for 120 classes (dog breeds)

## Build, Compile and Train the Model

In [ ]:
# Create the final model
# model = Model(inputs=base_model.input, outputs=x)
model = models.Model(inputs=base_model.input, outputs=x)

In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Define EarlyStopping callback to stop training early
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=30,               # Stop training if no improvement in 30 epochs
    verbose=1,                # Print a message when training is stopped early
    restore_best_weights=True # Restore the best model weights
)


# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[lr_scheduler, early_stopping]  # Use early stopping and learning rate scheduler
)

## Fine-tune the Model

In [ ]:
# Fine-tune the model by unfreezing some layers
for layer in base_model.layers[-150:]:  # Unfreeze the last 100 layers
    layer.trainable = True

In [ ]:
# Recompile the model after unfreezing layers
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
# Fine-tune the model with the updated optimizer and the same learning rate scheduler
history_fine_tune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=150,
    callbacks=[lr_scheduler, early_stopping] # Continue using the LR scheduler and early stopping
)

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
print(f"Validation Loss: {loss:.4f}")

## Plot training and validation accuracy/loss

In [ ]:
# Get the training and validation metrics from the history object
acc = history_fine_tune.history['accuracy']
val_acc = history_fine_tune.history['val_accuracy']
loss = history_fine_tune.history['loss']
val_loss = history_fine_tune.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(14, 7))

# Plot Training and Validation Loss with Different Colors
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, label='Training Loss', color='blue')
plt.plot(epochs, val_loss, label='Validation Loss', color='red')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot Training and Validation Accuracy with Different Colors
plt.subplot(1, 2, 2)
plt.plot(epochs, acc, label='Training Accuracy', color='green')
plt.plot(epochs, val_acc, label='Validation Accuracy', color='orange')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Save the trained model using the recommended Keras format
model.save('dog_breeds_resnet50_model.keras')

In [ ]:
# Load the saved model
loaded_model = tf.keras.models.load_model('dog_breeds_resnet50_model.keras')

In [ ]:
# Get true labels and predictions for the test dataset
Y_true_test = []
Y_pred_test = []
#
for images, labels in validation_generator:
    Y_true_test.extend(labels) # Collect true labels
    Y_pred_prob = model.predict(images, verbose=0) # Get predicted probabilities
    Y_pred_test.extend(np.argmax(Y_pred_prob, axis=1)) # Convert probabilities to predicted labels

# Calculate Accuracy, Precision, Recall, F1-score
accuracy = accuracy_score(Y_true_test, Y_pred_test)
precision = precision_score(Y_true_test, Y_pred_test, average='weighted')  # 'weighted' for multi-class classification
recall = recall_score(Y_true_test, Y_pred_test, average='weighted')
f1 = f1_score(Y_true_test, Y_pred_test, average='weighted')

# Print the evaluation metrics
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


In [ ]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_true_test, Y_pred_test)

# Create a heatmap of the confusion matrix
plt.figure(figsize=(12, 10))  # Adjust the figure size to fit the large matrix

# If you only want to focus on the top 10 most confusing pairs,
# you can slice your confusion matrix like so:
top_10_conf_matrix = conf_matrix[:10, :10]

sns.heatmap(top_10_conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix (Top 10 classes)')
plt.show()

## Test a Picture

In [ ]:
# Function to preprocess the image
def preprocess_image(image_path, target_size=(224, 224)):
    """
    Preprocess the image for prediction.
    Args:
        image_path: Path to the input image.
        target_size: Tuple specifying the input size for the model.
    Returns:
        Preprocessed image as a numpy array.
    """
    img = load_img(image_path, target_size=target_size)  # Load and resize the image
    img_array = img_to_array(img)  # Convert to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize pixel values
    return img_array

# Path to the test image
image_path = '/Users/pingsusu/Downloads/Dog 2.png'

# Preprocess the test image
input_image = preprocess_image(image_path)

# Make prediction
predictions = model.predict(input_image)
predicted_class_index = np.argmax(predictions)  # Get the index of the highest probability
predicted_class_name = class_names[predicted_class_index]  # Map to the breed name

# Display the image and prediction
plt.figure(figsize=(6, 6))
img = load_img(image_path)
plt.imshow(img)
plt.axis('off')
plt.title(f"Predicted Breed: {predicted_class_name}", fontsize=16)
plt.show()

print(f"The predicted breed is: {predicted_class_name}")
